# Overview

SWAT을 사용하여 CNN 모형을 적합

Server는 Kolon Server를 사용하였음

### 필요 라이브러리 호출

In [123]:
# library import
import os
import numpy as np
import scipy as sp
import pandas as pd
import tensorflow as tf

%matplotlib inline
import matplotlib.pyplot as plt

import cv2

In [2]:
import warnings
warnings.simplefilter("ignore", DeprecationWarning)

### Start CAS session

In [124]:
import swat

os.environ['CAS_CLIENT_SSL_CA_LIST'] = r'C:\Users\korbsl\OneDrive - SAS\Backup\800_Install\20_Python\trustedcerts.pem'

hostname = 'casc.kolon.com'
port = 8777
username = 'viya_sask01'
password = 'viya'

# Connect to CAS
sess = swat.CAS(hostname = hostname,
                port = port,
                username = username,
                password = password,
                protocol = 'https')
# serverStatus action in builtins action set
print(sess.builtins.serverStatus())
print(sess)


NOTE: Grid node action status report: 5 nodes, 9 total actions executed.
[About]

 {'CAS': 'Cloud Analytic Services',
  'Copyright': 'Copyright Â© 2014-2018 SAS Institute Inc. All Rights Reserved.',
  'ServerTime': '2019-04-09T01:49:17Z',
  'System': {'Hostname': 'casc',
   'Linux Distribution': 'Red Hat Enterprise Linux Server release 7.4 (Maipo)',
   'Model Number': 'x86_64',
   'OS Family': 'LIN X64',
   'OS Name': 'Linux',
   'OS Release': '3.10.0-693.el7.x86_64',
   'OS Version': '#1 SMP Thu Jul 6 19:56:57 EDT 2017'},
  'Version': '3.04',
  'VersionLong': 'V.03.04M0P07122018',
  'license': {'expires': '30Jun2019:00:00:00',
   'gracePeriod': 0,
   'maxCPUs': 100,
   'site': 'DEMO CENTER-VIYA',
   'siteNum': 70232805,
   'warningPeriod': 30}}

[nodestatus]

 Node Status
 
               name        role  uptime  running  stalled
 0  casw1.kolon.com      worker   3.757        0        0
 1  casw2.kolon.com      worker   3.757        0        0
 2  casw3.kolon.com      worker   3.757 

In [125]:
sess.builtins.serverstatus()

NOTE: Grid node action status report: 5 nodes, 10 total actions executed.


[About]

 {'CAS': 'Cloud Analytic Services',
  'Copyright': 'Copyright Â© 2014-2018 SAS Institute Inc. All Rights Reserved.',
  'ServerTime': '2019-04-09T01:49:21Z',
  'System': {'Hostname': 'casc',
   'Linux Distribution': 'Red Hat Enterprise Linux Server release 7.4 (Maipo)',
   'Model Number': 'x86_64',
   'OS Family': 'LIN X64',
   'OS Name': 'Linux',
   'OS Release': '3.10.0-693.el7.x86_64',
   'OS Version': '#1 SMP Thu Jul 6 19:56:57 EDT 2017'},
  'Version': '3.04',
  'VersionLong': 'V.03.04M0P07122018',
  'license': {'expires': '30Jun2019:00:00:00',
   'gracePeriod': 0,
   'maxCPUs': 100,
   'site': 'DEMO CENTER-VIYA',
   'siteNum': 70232805,
   'warningPeriod': 30}}

[nodestatus]

 Node Status
 
               name        role  uptime  running  stalled
 0  casw1.kolon.com      worker   7.400        0        0
 1  casw2.kolon.com      worker   7.400        0        0
 2  casw3.kolon.com      worker   7.400        0        0
 3  casw4.kolon.com      worker   7.400        0        0
 4   casc.kolon.com  controller   7.444        0        0

[server]

 Server Status
 
    nodes  actions
 0      5       10

+ Elapsed: 0.00576s, user: 0.000862s, sys: 0.00504s, mem: 1.37mb

### CAS Library 생성

In [126]:
info = sess.table.caslibInfo()
if info.severity == 0 and any(info.CASLibInfo['Name'].isin(['imagelib'])):
    sess.table.dropcaslib(caslib='imagelib')
    
sess.table.addcaslib(name='imagelib', datasource={'srcType':'path'}, path='/home/viya_sask01/korbsl/VDMML_training/data/image_data', 
                     subdirectories=True)

NOTE: 'imagelib' is now the active caslib.
NOTE: Cloud Analytic Services added the caslib 'imagelib'.


,Name,Type,Description,Path,Definition,Subdirs,Local,Active,Personal,Hidden,Transient
0,imagelib,PATH,,/home/viya_sask01/korbsl/VDMML_training/data/i...,,1.0,1.0,1.0,0.0,0.0,0.0


### Load Actionset

In [127]:
sess.loadactionset('sampling')
sess.loadactionset('image')

sess.loadactionset('deeplearn')

NOTE: Added action set 'sampling'.
NOTE: Added action set 'image'.
NOTE: Added action set 'deeplearn'.


[actionset]

 'deeplearn'

+ Elapsed: 0.00981s, user: 0.00584s, sys: 0.0113s, mem: 0.99mb

### Load Image data into CAS

In [128]:
r1 = sess.image.loadimages(caslib='imagelib', path='CIFAR10/train', decode=True, recurse=True, labellevels=-1,
                     addcolumns='CHANNELCOUNT', casout=dict(name='CIFAR10_train',replace=True))
CIFAR10_train = sess.CASTable(caslib='imagelib', name='CIFAR10_train')

r2 = sess.image.loadimages(caslib='imagelib', path='CIFAR10/test', decode=True, recurse=True, labellevels=-1,
                     addcolumns='CHANNELCOUNT', casout=dict(name='CIFAR10_test',replace=True))
CIFAR10_test = sess.CASTable(caslib='imagelib', name='CIFAR10_test')


NOTE: Loaded 50000 images from /home/viya_sask01/korbsl/VDMML_training/data/image_data/CIFAR10/train into Cloud Analytic Services table CIFAR10_train.
NOTE: Loaded 10000 images from /home/viya_sask01/korbsl/VDMML_training/data/image_data/CIFAR10/test into Cloud Analytic Services table CIFAR10_test.


### Sampling data

In [129]:
with CIFAR10_train as dat:
    dat.groupby={'_label_'}
    dat.sampling.stratified(
        partInd=True,
        samppct=80,
        samppct2=20,
        seed=12345,
        output={
            "casout":{"name":"CIFAR10_train_part","replace":True},
            "copyVars":"all"
        }
    )
    
CIFAR10_train_part = sess.CASTable('CIFAR10_train_part')

NOTE: Using SEED=12345 for sampling.


In [131]:
ci = CIFAR10_train_part.columninfo()
display(ci)
type(CIFAR10_train_part)

,Column,Label,ID,Type,RawLength,FormattedLength,NFL,NFD
0,_dimension_,,1,int64,8,12,0,0
1,_resolution_,,2,varbinary,16,16,0,0
2,_imageFormat_,,3,int64,8,12,0,0
3,_channelCount_,,4,int64,8,12,0,0
4,_image_,,5,varbinary,3072,3072,0,0
5,_size_,,6,int64,8,12,0,0
6,_path_,,7,varchar,115,115,0,0
7,_label_,,8,varchar,10,10,0,0
8,_type_,,9,char,3,3,0,0
9,_id_,,10,int64,8,12,0,0


swat.cas.table.CASTable

In [10]:
CIFAR10_train_part['_label_'].groupby('_PartInd_').value_counts()

_PartInd_  _label_   
1.0        airplane      4000
           automobile    4000
           bird          4000
           cat           4000
           deer          4000
           dog           4000
           frog          4000
           horse         4000
           ship          4000
           truck         4000
2.0        airplane      1000
           automobile    1000
           bird          1000
           cat           1000
           deer          1000
           dog           1000
           frog          1000
           horse         1000
           ship          1000
           truck         1000
dtype: int64

In [134]:
sess.table.shuffle(table='CIFAR10_train_part',
                  casout=dict(name='CIFAR10_train_shuffle', replace=True))
CIFAR10_train_shuffle = sess.CASTable('CIFAR10_train_shuffle')

In [135]:
sess.fetch(CIFAR10_train_shuffle, to=5)

,_dimension_,_resolution_,_imageFormat_,_channelCount_,_image_,_size_,_path_,_label_,_type_,_id_,_PartInd_
0,2,b' \x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x...,16,3,b'\x94\x87w\x97\x8b{\x98\x8b{\x8e\x81q\x8c\x7f...,3072,/home/viya_sask01/korbsl/VDMML_training/data/i...,truck,png,48164,2.0
1,2,b' \x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x...,16,3,b'\xc5\xb6\xb4\xc4\xb5\xb3\xc3\xb4\xb2\xc2\xb3...,3072,/home/viya_sask01/korbsl/VDMML_training/data/i...,ship,png,40796,1.0
2,2,b' \x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x...,16,3,"b""\xff\xff\xff\xf0\xf0\xf0\xef\xef\xef\xf5\xf5...",3072,/home/viya_sask01/korbsl/VDMML_training/data/i...,ship,png,41539,1.0
3,2,b' \x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x...,16,3,b'\xe0\xbb\x9f\xdd\xb8\x9c\xdd\xb9\x9d\xdd\xb9...,3072,/home/viya_sask01/korbsl/VDMML_training/data/i...,airplane,png,1693,1.0
4,2,b' \x00\x00\x00\x00\x00\x00\x00 \x00\x00\x00\x...,16,3,b'\x9a\xb4\xba\x98\xb2\xb9\xac\xc0\xc5\xb2\xc5...,3072,/home/viya_sask01/korbsl/VDMML_training/data/i...,dog,png,27572,1.0


In [136]:
sess.image.summarizeimages(table=dict(caslib='imagelib',name='CIFAR10_train_shuffle',where='_PartInd_=1'))

NOTE: Table CIFAR10_TRAIN_SHUFFLE contains decompressed images.
NOTE: 40000 out of 40000 images were scanned and results were saved to the summary table.


,Column,png,minWidth,maxWidth,minHeight,maxHeight,meanWidth,meanHeight,mean1stChannel,min1stChannel,max1stChannel,mean2ndChannel,min2ndChannel,max2ndChannel,mean3rdChannel,min3rdChannel,max3rdChannel
0,_image_,40000.0,32.0,32.0,32.0,32.0,32.0,32.0,113.822313,0.0,255.0,122.945872,0.0,255.0,125.257806,0.0,255.0


### Building CNN Model Architecture

In [137]:
sess.buildmodel(model=dict(name='ConvNet', replace=True), type='CNN')

sess.addlayer(model='ConvNet', name='input_layer', 
              layer=dict(type='input', nchannels=3, width=32, height=32, std='std', offsets={113.822313,122.945872,125.257806}))
sess.addlayer(model='ConvNet', name='conv_layer1',
              layer=dict(type='CONVO', nFilters=32, width=5, height=5, stride=1), srcLayers='input_layer')
sess.addlayer(model='ConvNet', name='pool_layer1',
              layer=dict(type='POOL', pool='max', width=2, height=2, stride=2), srcLayers='conv_layer1')

sess.addlayer(model='ConvNet', name='conv_layer2',
              layer=dict(type='CONVO', act='Identity', nFilters=32, width=5, height=5, stride=1), srcLayers='pool_layer1')
sess.addlayer(model='ConvNet', name='batch_layer1',
              layer=dict(type='BATCHNORM', act='RELU'), srcLayers='conv_layer2')
sess.addlayer(model='ConvNet', name='pool_layer2',
              layer=dict(type='POOL', pool='max', width=2, height=2, stride=2), srcLayers='batch_layer1')

sess.addlayer(model='ConvNet', name='fc_layer1',
              layer=dict(type='FULLCONNECT', n=20, act='Identity', init='xavier', dropout=0.2, includeBias=False), srcLayers='pool_layer2')
sess.addlayer(model='ConvNet', name='batch_layer2',
              layer=dict(type='BATCHNORM', act='RELU'), srcLayers='fc_layer1')
sess.addlayer(model='ConvNet', name='output_layer',
              layer=dict(type='OUTPUT', act='SOFTMAX'), srcLayers='batch_layer2')

,casLib,Name,Rows,Columns,casTable
0,imagelib,convnet,110,5,"CASTable('convnet', caslib='imagelib')"


In [138]:
sess.tableInfo()

,Name,Rows,Columns,IndexedColumns,Encoding,CreateTimeFormatted,ModTimeFormatted,AccessTimeFormatted,JavaCharSet,CreateTime,...,Global,Repeated,View,SourceName,SourceCaslib,Compressed,Creator,Modifier,SourceModTimeFormatted,SourceModTime
0,CIFAR10_TRAIN,50000,10,0,utf-8,2019-04-09T01:49:39+00:00,2019-04-09T01:49:39+00:00,2019-04-09T01:50:13+00:00,UTF8,1.870394e+09,...,0,0,0,,,0,viya_sask01,,,NaN
1,CIFAR10_TEST,10000,10,0,utf-8,2019-04-09T01:49:42+00:00,2019-04-09T01:49:42+00:00,2019-04-09T01:49:42+00:00,UTF8,1.870394e+09,...,0,0,0,,,0,viya_sask01,,,NaN
2,CIFAR10_TRAIN_PART,50000,11,0,utf-8,2019-04-09T01:50:13+00:00,2019-04-09T01:50:13+00:00,2019-04-09T01:53:46+00:00,UTF8,1.870394e+09,...,0,0,0,,,0,viya_sask01,,,NaN
3,CIFAR10_TRAIN_SHUFFLE,50000,11,0,utf-8,2019-04-09T01:53:46+00:00,2019-04-09T01:53:46+00:00,2019-04-09T01:54:26+00:00,UTF8,1.870394e+09,...,0,0,0,,,0,viya_sask01,,,NaN
4,CONVNET,110,5,0,utf-8,2019-04-09T01:56:13+00:00,2019-04-09T01:56:13+00:00,2019-04-09T01:56:13+00:00,UTF8,1.870394e+09,...,0,0,0,,,0,viya_sask01,,,NaN


### deeplearning Training

In [21]:
sess.dlTrain(model='ConvNet',
             table=dict(caslib='imagelib', name='CIFAR10_TRAIN_SHUFFLE', where='_PartInd_=1'),
             modelWeights=dict(name='CIFAR10_TRAIN_mwgt', replace=True),
             bestWeights=dict(name='CIFAR10_TRAIN_bwgt', replace=True),
             inputs='_image_',
             target='_label_',
             nominal='_label_',
             ValidTable=dict(caslib='imagelib', name='CIFAR10_TRAIN_SHUFFLE', where='_PartInd_=2'),
             
             optimizer=dict(minibatchsize=60,
                            algorithm={'method':'ADAM', 'lrpolicy':'Step', 'gamma':0.5, 'learningrate':0.001},
                            maxepochs=80),
             seed=12345
            )

,Descr,Value
0,Model Name,convnet
1,Model Type,Convolutional Neural Network
2,Number of Layers,9
3,Number of Input Layers,1
4,Number of Output Layers,1
5,Number of Convolutional Layers,2
6,Number of Pooling Layers,2
7,Number of Fully Connected Layers,1
8,Number of Batch Normalization Layers,2
9,Number of Weight Parameters,69160


### Scoring test Data

In [22]:
sess.dlScore(modelTable=dict(caslib='imagelib', name='ConvNet'),
             initWeights=dict(caslib='imagelib', name='CIFAR10_TRAIN_bwgt'),
             table=dict(caslib='imagelib', name='CIFAR10_test'),
             casout=dict(caslib='imagelib', name='CIFAR10_scored', replace=True),
             copyVars='_label_',
             
             layerOut=dict(name='CIFAR10_Layer_data', replace=True),
             layers='conv_layer1',
             layerImageType='JPG'
            )

[OutputCasTables]

      casLib                Name   Rows  Columns  \
 0  imagelib  CIFAR10_Layer_data  10000       33   
 1  imagelib      CIFAR10_scored  10000       14   
 
                                             casTable  
 0  CASTable('CIFAR10_Layer_data', caslib='imagelib')  
 1      CASTable('CIFAR10_scored', caslib='imagelib')  

[ScoreInfo]

                          Descr         Value
 0  Number of Observations Read         10000
 1  Number of Observations Used         10000
 2  Misclassification Error (%)         40.74
 3                   Loss Error      1.295668

+ Elapsed: 1.18s, user: 22.4s, sys: 3.74s, mem: 1.34e+03mb

In [25]:
sess.table.fetch(table='CIFAR10_scored', to=5)

,_label_,_DL_P0_,_DL_P1_,_DL_P2_,_DL_P3_,_DL_P4_,_DL_P5_,_DL_P6_,_DL_P7_,_DL_P8_,_DL_P9_,_DL_PredName_,_DL_PredP_,_DL_PredLevel_
0,airplane,0.084796,0.035257,0.348475,0.063517,0.050350,0.029570,0.061706,0.020894,0.081091,0.224345,ship,0.348475,2.0
1,airplane,0.087705,0.061635,0.294032,0.042137,0.027873,0.210369,0.044704,0.049856,0.116744,0.064945,ship,0.294032,2.0
2,airplane,0.028996,0.018988,0.052040,0.074548,0.014796,0.744827,0.007001,0.012079,0.012678,0.034046,airplane,0.744827,5.0
3,airplane,0.138917,0.069154,0.218287,0.053081,0.047764,0.138614,0.094436,0.100351,0.067532,0.071865,ship,0.218287,2.0
4,airplane,0.032014,0.030929,0.209128,0.070468,0.020470,0.508413,0.012479,0.016196,0.036612,0.063291,airplane,0.508413,5.0


### Assess Models

In [37]:
sess.loadactionset('percentile')
    

NOTE: Added action set 'percentile'.


[actionset]

 'percentile'

+ Elapsed: 0.00607s, user: 0.00281s, sys: 0.0036s, mem: 1.06mb

In [39]:
CIFAR10_scored = sess.CASTable('CIFAR10_scored')


In [121]:
a = sess.crosstab(table='CIFAR10_scored', row='_DL_PredName_',col='_label_')
a.rename(index=str, columns={'Col1':airplane})

AttributeError: 'CASResults' object has no attribute 'rename'

In [115]:
b = CIFAR10_scored[['_DL_PredLevel_', '_DL_PredName_']].values

In [116]:
np.vstack({tuple(e) for e in b})

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  """Entry point for launching an IPython kernel.


array([['2.0', 'ship'],
       ['4.0', 'frog'],
       ['7.0', 'dog'],
       ['1.0', 'horse'],
       ['3.0', 'automobile'],
       ['6.0', 'cat'],
       ['0.0', 'truck'],
       ['9.0', 'bird'],
       ['8.0', 'deer'],
       ['5.0', 'airplane']], dtype='<U32')

### Session 종료

In [4]:
# sess.terminate()